# DANTE.py Figure Maker

All of the plots that require more than one data file will be made in this notebook.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
import matplotlib.lines as mlines
import seaborn as sns
import pandas as pd